### MATH2504 S2, 2022, BigHW Submission 
### Submission form IV of IV 

Student #1 name: Rachel Dudley

Student #1 ID: 46982243

Student #2 name: Wilson Wang

Student #2 ID: 46472856

GitHub repo: Rachel-Dudley__Wilson-Wang-2504-2022-BigHW

---

**Note: It is expected that this notebook be runnable in the sense that the marker can select "Restart & Run All" and all code cells of solutions will execute. This requires not to have name clashes between solutions.**

---

# Solution to [Question 13](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q13)

## 13a:

In [ ]:
using Plots

diff_forward(a, x; h = sqrt(eps())) = (a(x+h) - a(x))/h
a(x) = sin(x^2)
a_der(x) = 2x*cos(x^2) 

diff_forward(b, x; h = sqrt(eps())) = (b(x+h) - b(x))/h
b(x) = exp(x)
b_der(x) = exp(x)

diff_forward(c, x; h = sqrt(eps())) = (c(x+h) - c(x))/h
c(x) = atan(x)/(1+exp(-x^2))
c_der(x) = (2*x*exp(-x^2)*atan(x))/((exp(-x^2) + 1)^2) + 1/((x^2 + 1) * (exp(-x^2) + 1))

x0, x1, x2 = 0.5, 1, 2
h_range = 10 .^ (-17:0.01:1)
errs0 = [abs(diff_forward(a, x0; h = h) - a_der(x0))/abs(a_der(x0)) for h in h_range]
errs1 = [abs(diff_forward(b, x1; h = h) - b_der(x1))/abs(b_der(x1)) for h in h_range]
errs2 = [abs(diff_forward(c, x2; h = h) - c_der(x2))/abs(c_der(x2)) for h in h_range]
plot(h_range,[errs0 errs1 errs2], yaxis = :log,xaxis = :log,
    xlabel="h",ylabel="Absolute relative error",
    label = ["x = $x0" "x = $x1" "x = $x2"], c = [:red :blue :black], legend = :bottomright, title="Forward Difference Scheme Performance")

In [ ]:
@show findmin(errs0)
@show findmin(errs1)
@show findmin(errs2)

@show h_range[851]
@show h_range[857]
@show h_range[929]

The optimal h value, is the value where the error is the smallest. The optimal h values for each function for the forward difference scheme are as follows:

                sin(x^2) at x=1/2: h = 3.1622776601683795e-9
                       e^x at x=1: h = 3.6307805477010177e-9
    arctan(x)/(1+e^(-x^2)) at x=2: h = 1.9054607179632482e-8


In [ ]:
diff_central(a, x; h = sqrt(eps())) = (a(x+(h/2)) - a(x-(h/2)))/h
diff_central(b, x; h = sqrt(eps())) = (b(x+(h/2)) - b(x-(h/2)))/h
diff_central(c, x; h = sqrt(eps())) = (c(x+(h/2)) - c(x-(h/2)))/h
    
errors0 = [abs(diff_central(a, x0; h = h) - a_der(x0))/abs(a_der(x0)) for h in h_range]
errors1 = [abs(diff_central(b, x1; h = h) - b_der(x1))/abs(b_der(x1)) for h in h_range]
errors2 = [abs(diff_central(c, x2; h = h) - c_der(x2))/abs(c_der(x2)) for h in h_range]
plot(h_range,[errors0 errors1 errors2], yaxis = :log,xaxis = :log,
    xlabel="h",ylabel="Absolute relative error",
    label = ["x = $x0" "x = $x1" "x = $x2"], c = [:red :blue :black], legend = :bottomright, title="Central Difference Scheme Performance")

In [ ]:
@show findmin(errors0)
@show findmin(errors1)
@show findmin(errors2)

@show h_range[1147]
@show h_range[1165]
@show h_range[1200];

The optimal h values for each function for the central difference scheme are as follows:

                sin(x^2) at x=1/2: h = 2.884031503126606e-6
                       e^x at x=1: h = 4.365158322401657e-6
    arctan(x)/(1+e^(-x^2)) at x=2: h = 9.772372209558111e-6
    
Compared to the FDS, these h values are further away from zero which means there become issues because of the floating point values earlier on for the CDS. This makes sense because the formula for the CDS involves more operations being applied to the float64 h values, which creates more extra decimals (i.e. h/2 is in this formula 2 times which causes more issues)

## 13b:

In [ ]:
# Convert to a markdown cell and answer 13b here (in place of this comment)

## 13c:

In [ ]:
### Answer 13b here (in place of this comment)

---

*Use this area below as an experimental playground for the question. Feel free to leave some of the experimental code here with your submision. However, it will not be marked or reviewed. Make sure in your submission that if there is any experimental code here then running it does not interfere with running the actual solutions.*

---

# Solution to [Question 14](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q14)

In [ ]:
#Method 1
#care that I - P is singular that means that there is no inverse to the matrix
using LinearAlgebra
#This function creates LxL dimension matrix P
function structured_P(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)::Matrix{Float64}
    q = 1 - p - r
    P = diagm(fill(r,L)) + diagm(-1=>fill(q,L-1)) + diagm(1 => fill(p,L-1))
    P[1,1] = 1-p
    P[L,L] = 1-q
    return P
end
# creaats L vector pi that matches P
structured_π(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)::Vector{Float64} = begin
    q = 1 - p - r
    [(p/q)^i  for i in 1:L] * (q-p) / p / (1-(p/q)^L) #Explicit expression (birth death)
end;

function Linear_op(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)
    A = vcat(structured_P(L), ones(1,L)) #A is matrix added another row of ones to P
    b = vcat(structured_π(L), ones(1,1)) #b is the pi vector and add another 1 at the bottom
    π_origin = structured_π(L)
    #This allows a linear system to be determined
    #Let x be the solution to estimate pi 
    x = \(A, b) #This allow the estimation of pi from solving linear system 
    #finding the least square between the two
    # @show (π_origin .- x).^2
return x
# sum((π_origin .- x).^2)
end 
for L ∈ [2, 3, 5, 10, 20, 30, 40, 50, 100, 200, 300, 400, 500, 1000]
   @show Linear_op(L)
   @show norm(Linear_op(L) - structured_π(L))
end

#Method 2
function Limits_op(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)
    n=10^10 #define a really large number 
    # π_leg = Array{Float64}(undef, L)
    π_est = Array{Float64}(undef, L)
    P_n = (structured_P(L)')^n #This line give the matrix to the power clsoe to infinity
    for i ∈ 1:L
        π_est[i] = P_n[i,1]
        # π_leg[i] = structured_π(L)[i] #since all the rows values will converge to the same thing
    end 
    return π_est
end 
for L ∈ [2, 3, 5, 10, 20, 30, 40, 50, 100, 200, 300, 400, 500, 1000]
    @show Limits_op(L)
    @show norm(Limits_op(L)-structured_π(L))
 end# @show Limits_op(5)
# @show sum(P_n, dims = 1) #sum of column

#method 3
function eigen_π(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)
    e_vector = Array{Float64}(undef, L)
    cor_vector = Array{Vector{Float64}}(undef, L)
    d_repeat = repeat([1], L)
    for i ∈ 1:L
        e_vector[i] = eigvecs(structured_P(L))[i]
        @show e_vector
        if structured_P(L)*(e_vector[i]) == Diagonal(d_repeat)*(e_vector[i]) 
            cor_vector = push!(cor_vector, e_vector[j])
        end
    # @show Diagonal(d_repeat)*(e_vector[i])
   
    # @show eigvecs(structured_P(L)[1 , eigvals])
    end
    return cor_vector 
end
@show eigen_π(2)

---

*Use this area below as an experimental playground for the question. Feel free to leave some of the experimental code here with your submision. However, it will not be marked or reviewed. Make sure in your submission that if there is any experimental code here then running it does not interfere with running the actual solutions.*

---

# Solution to [Question 15](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q15)

## 15a:

In [ ]:
state_v_x(u::Vector{Float64}) = u[1]
state_v_y(u::Vector{Float64}) = u[2]
state_x(u::Vector{Float64}) = u[3]
state_y(u::Vector{Float64}) = u[4]

function df_dt_one_body(u::Vector{Float64}, t::Float64)::Vector{Float64}
    M, G = 1, 1 #We take these constants as normalized. Naturally they would need to be set for physical values.
    r = sqrt(state_x(u)^2 + state_y(u)^2)
    return [-M*G*state_x(u)/r^3, -M*G*state_y(u)/r^3, state_v_x(u), state_v_y(u)]
end

In [ ]:
using Plots, Measures

function plot_solution( t::AbstractArray{T}, 
                        u::Vector{Vector{Float64}}; 
                        title::String = "",
                        label::Union{String, Bool} = false) where T
    x, y, v_x, v_y = state_x.(u), state_y.(u), state_v_x.(u), state_v_y.(u)
 
    #"Energy"
    r = @. sqrt(x^2 + y^2)
    E = @. 0.5*(v_x^2 + v_y^2) - 1.0/r

    p1 = plot(  x, y, label = label, xlabel= "X", ylabel = "Y",
                title = title*" (position)", aspectratio=1,legend=:topleft,ylim=(-7,7))
    scatter!([0], [0], ms=15, msw=0, c=:orange, shape =:star, label="Sun")
    scatter!([x[1]], [y[1]], ms=5, msw=0, c=:blue, shape =:circle, label="Earth initial position")

    p4 = plot(  t, E, xlabel = "Time", ylabel = "Energy",
                label = label, title = title*" (energy)")
    plot(p1, p4, margin = 10mm,size=(800,400))
end

In [ ]:
using Serialization, HTTP

h1, h2 = 0.01, 0.001
t1 = 0: h1 : 200
u_0 = [0., 1, 1.5, 0]

#Note: the data was written to file via serialize("data/example_trajectory.dat", u) and then committed to GitHub
ode_web = HTTP.request("GET","https://github.com/yoninazarathy/"*
                            "ProgrammingCourse-with-Julia-SimulationAnalysisAndLearningSystems/"*
                            "raw/e71c6d085e0fcf79e3737c3a1db0fc56904e3f10/data/example_trajectory.dat")
u = deserialize(IOBuffer(ode_web.body))
@show typeof(u)
plot_solution(t1,u,title = "Euler's method", label="h1=$h1 h2=$h2")

## 15b:

In [ ]:
### Answer 15b here (in place of this comment)

## 15c:

for Eulers method, it predicts the trajectory better at smaller values of h. but floating point errors will still mean it isnt going to be the most accurate if it is very very close to 0.

## 15d:

In [ ]:
### Answer 15d here (in place of this comment)

---

*Use this area below as an experimental playground for the question. Feel free to leave some of the experimental code here with your submision. However, it will not be marked or reviewed. Make sure in your submission that if there is any experimental code here then running it does not interfere with running the actual solutions.*

---

# Solution to [Question 16](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q16)

## 16a:

In [ ]:
### Answer 16a here (in place of this comment)

## 16b:

In [ ]:
### Answer 16b here (in place of this comment)

## 16c:

In [ ]:
### Answer 16c here (in place of this comment)

## 16d:

In [ ]:
### Answer 16d here (in place of this comment)

---

*Use this area below as an experimental playground for the question. Feel free to leave some of the experimental code here with your submision. However, it will not be marked or reviewed. Make sure in your submission that if there is any experimental code here then running it does not interfere with running the actual solutions.*

---

# Solution to [Question 17](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q17)

Amy Chan’s perspective seminar was very interesting and useful to hear about, especially as she is working in a field that I and many people in our course are probably considering moving towards in our futures. Throughout her talk, Chan spoke about her path to becoming where she is now, about her current company and what she likes about it there, the different experiences within her career she has had, and she also shared some tips from what she has learnt and thought would be helpful for us to hear.

Amy Chan studied at UQ, starting with an Undergrad and then doing her Honours, during which she did a lot of analysis which was a big part of her studies. She then took up an opportunity at CSIRO doing image analysis which included projects like: she was given random pictures of a huge wheat field which all looked almost exactly the same in the photos, but she had to use her analysis to figure out how to piece the pictures together in the correct order. To do this she used many procedures, but I think a very interesting thing she had to consider was the angle the photo was taken at from the sky and how that matches the other images. After this, Chan went on to do a PhD. where she decided to learn statistics as it would be useful to learn more of… and after all of this she found out that research is not for her because she doesn’t enjoy it as much as the practical side of things.

From this realisation that it was time to leave academia and research, Chan went to a company called Polymathian which is a company that takes client problems and uses software to make solutions which they can sell. She has been here ever since completing her studies and seems to be very happy there. It has brought her a lot of opportunities to move around within different fields of the company e.g. the different parts Chan has worked in include RACE, SOLO, ORB and more. When she talked about the tool RACE, I found it very interesting the sorts of problems they find solutions to. Chan explained a task which initially seemed quite simple: getting a train to go somewhere to transport wheat. It then became clear after hearing a few difficulties with this, that there are so many variables and constraints to consider here, like: What is the demand of wheat? What timetables do the trains run on? How many trains are running at a time? What networks do the trains run on? When does the transportation have to be done by? How much does everything cost?... and these are just SOME of the considerations.

Polymathian is a small company with not very many employees working there, but this is something Amy Chan views as a positive thing. It has not only lead her to gain lots of experience in different areas of the industry, however it has also provided benefits in terms of the social and lifestyle aspects. For example the team at Polymathian (pre-covid) travelled together doing fun things like skiing. As well as this they go hiking together and other activities which are not inly fun, but also make the team closer and potentially boosts their ability to work effectively together.

Some key tools that Chan spoke about were Excel – a very prominent software which is very widely used by almost everyone so Excel is something to get used to, Python – which is a very commonly used language used for the backend. Pythin is also easy to learn and use and is free (which is a big benefit compared to software like MATLAB which is very expensive). Other tools mentioned were React and Javascript – used for frontend things, and AWS (Amazon web service) & Docker, which I had not heard of before but they are used to deliver apps over the web.

From the talk, Amy gave a lot of very helpful tips that I will definitely take on and remember. The first piece of advice was to do internships and take opportunities that come your way. This is a very useful, quick and practical way of finding out what you like/don’t like, and what sort of things you could be heading towards doing for a job. Another tip that is very good to know is that just because a job has components to do with something you have got very limited knowledge in, don’t let that turn you away. You learn so much on the job, and the best way to learn is to get involved and emersed in it. This specific piece of advice is one that I am currently learning is very true through this course MATH2504. What I knew 5 weeks ago about computers and coding compared with now is massively different and that came from getting stuck into the big homework which was completely outside my knowledge domain.

Overall, Amy Chan’s talk was very good to hear, and it has made me want to go and find lots of ways of learning new things, and to discover internships and similar opportunities which could help me to become more knowledgeable, employable, and know what I want from my future.
